In [1]:
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


#Importing Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Importing Data from Wikipedia To Test

In [3]:
from html.parser import HTMLParser
response=requests.get(URL)
Soup=BeautifulSoup(response.text,'html.parser')
#print(Soup)
#response.text


In [4]:
table=Soup.find('table', {'class':'wikitable sortable'}).tbody                      
#Soup.table                                      #find('table', {'table class':'"wikitable sortable jquery-tablesorter"'}) #.t
#print(table)

In [5]:
rows=table.find_all('tr')
columns=[v.text.replace('\n','') for v in rows[0].find_all('th')]
print(columns)

['Postcode', 'Borough', 'Neighbourhood']


Converting the data into required Pandas DataFrame Format

In [6]:
df=pd.DataFrame(columns=columns)
for i in range(1,len(rows)):
    tds=rows[i].find_all('td')
    if len(tds)==4:
        values=[tds[0].text,tds[1].text,tds[2].text,tds[3].text.replace('\n','')]
    else:
        values=[td.text.replace('\n','') for td in tds]
    #print(values)
    df=df.append(pd.Series(values,index=columns),ignore_index=True)
    #break;


In [7]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Removing Not Assigned Values in Borough

In [8]:
#for i in range(0,287)
#  if 
df=df[(df['Borough']!="Not assigned")]  #& (df['Neighbourhood']!="Not assigned")]
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Checking Different Methods to Group Data to Required form
#df.groupby(['Postcode']).agg({'Borough':'first','Neighbourhood':lambda col: ','.join(col)})
df.groupby(['Postcode']).agg({'Neighbourhood':lambda col: ','.join(col),'Borough':'first'})
#df.groupby('Postcode')['Neighbourhood'].apply(lambda x: "{%s}" % ', '.join(x))
#df.groupby(['Postcode'])['Neighbourhood'].agg(lambda col: ','.join(col))
df.replace(to_replace="Not assigned",value="Queen's Park")   ## this will replace "Boston Celtics" with "Omega Warrior" 
#df.replace({'Neighbourhood': "Not assigned"}, {'Bourough'})

In [9]:
df=df.groupby(['Postcode']).agg({'Borough':'first','Neighbourhood':lambda col: ','.join(col)})
df.head()

,Neighbourhood,Borough
Postcode,,
M1B,"Rouge,Malvern",Scarborough
M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
M1E,"Guildwood,Morningside,West Hill",Scarborough
M1G,Woburn,Scarborough
M1H,Cedarbrae,Scarborough


In [10]:
#df.drop(['level_0'],axis=1,inplace=True)
df=df.reset_index()
#df.drop(['index'],axis=1,inplace=True)
df.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge,Malvern",Scarborough
1,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
2,M1E,"Guildwood,Morningside,West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


Shape of the DataFrame

In [11]:
df.shape

(103, 3)

### DataFrame Required For First Assignment

In [13]:
df.head(12)

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge,Malvern",Scarborough
1,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
2,M1E,"Guildwood,Morningside,West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park,Ionview,Kennedy Park",Scarborough
7,M1L,"Clairlea,Golden Mile,Oakridge",Scarborough
8,M1M,"Cliffcrest,Cliffside,Scarborough Village West",Scarborough
9,M1N,"Birch Cliff,Cliffside West",Scarborough


Reading the required Data From Csv file and Combining with our data

In [14]:
df1=pd.read_csv("https://cocl.us/Geospatial_data")

In [15]:
result=pd.concat([df, df1], axis=1, join='inner')

In [16]:
result.head()

,Postcode,Neighbourhood,Borough,Postal Code,Latitude,Longitude
0,M1B,"Rouge,Malvern",Scarborough,M1B,43.806686,-79.194353
1,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough,M1C,43.784535,-79.160497
2,M1E,"Guildwood,Morningside,West Hill",Scarborough,M1E,43.763573,-79.188711
3,M1G,Woburn,Scarborough,M1G,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,M1H,43.773136,-79.239476


In [17]:
df2=result[['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]

### The DataFrame That is required for  Second Assignment

In [19]:
df2.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


Using Geolocator to Find Latitude and Longitude of Toronto

In [20]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


Importing Folium Pacakage

In [21]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium
neighborhoods=df2


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

Map of Toronto with Neighbourhoods Marked on it 

In [24]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough']):
    label = '{}'.format( borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Taking ScarBorough Data  For Clustering Neighborhoods

In [25]:
Toronto_data = neighborhoods[neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
Toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [26]:
address = 'Scarborough,Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.773077, -79.257774.


Map of Borough Scarborough with other Borough near by it

In [27]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(Toronto_data['Latitude'], Toronto_data['Longitude'],Toronto_data['Borough']):
    label = '{}'.format( borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Foursquare ApI call to Get the required data of Venues available around the Scarborough

In [30]:
# @hidden_cell
CLIENT_ID = 'PHTU1TD0LXKS0VSI551NIYPTQDM4L1IKSX0CBW2MFZ2J1C1V' # your Foursquare ID
CLIENT_SECRET = 'NW1OC5XNNGI3RFFA4VKRE0HG0UNF0MCYMUHCAXXSKOFMBZTQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PHTU1TD0LXKS0VSI551NIYPTQDM4L1IKSX0CBW2MFZ2J1C1V
CLIENT_SECRET:NW1OC5XNNGI3RFFA4VKRE0HG0UNF0MCYMUHCAXXSKOFMBZTQ


In [31]:
Toronto_data.loc[0, 'Neighbourhood']

'Rouge,Malvern'

In [32]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


In [33]:
LIMIT=100;
radius=500;
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=PHTU1TD0LXKS0VSI551NIYPTQDM4L1IKSX0CBW2MFZ2J1C1V&client_secret=NW1OC5XNNGI3RFFA4VKRE0HG0UNF0MCYMUHCAXXSKOFMBZTQ&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

Converting Json Data Obtained From Foursquare into Pandas Dataframe for analysis

In [34]:
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Wendy's,"[{'name': 'Fast Food Restaurant', 'pluralName'...",43.807448,-79.199056


Cleaning the Data Obtained

In [35]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [37]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


Function to Find Neighbouring Places

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
Scarborough_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude'],radius=500
                                  )


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


Data Of Nearby venues

In [42]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(86, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [43]:
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea,Golden Mile,Oakridge",10,10,10,10,10,10
"Clarks Corners,Sullivan,Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest,Cliffside,Scarborough Village West",2,2,2,2,2,2
"Dorset Park,Scarborough Town Centre,Wexford Heights",8,8,8,8,8,8
"East Birchmount Park,Ionview,Kennedy Park",4,4,4,4,4,4


In [44]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


Applying Getdummies method for Statistical Analysis

In [46]:
# one hot encoding
Scar_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scar_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scar_onehot.columns[-1]] + list(Scar_onehot.columns[:-1])
Scar_onehot = Scar_onehot[fixed_columns]

Scar_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Gym,Hakka Restaurant,History Museum,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
Scar_onehot.shape

(86, 56)

In [48]:
Scar_grouped = Scar_onehot.groupby('Neighborhood').mean().reset_index()
Scar_grouped.head()


,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Gym,Hakka Restaurant,History Museum,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.0,0.0,0.00,0.0,0.000000,0.0,0.25,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.0,0.0
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Birch Cliff,Cliffside West",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.25,0.0,0.000000,0.0,0.00,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.0,0.0
3,Cedarbrae,0.0,0.142857,0.0,0.142857,0.142857,0.0,0.00,0.0,0.0,0.00,0.0,0.142857,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.00,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.142857,0.0,0.0
4,"Clairlea,Golden Mile,Oakridge",0.0,0.000000,0.0,0.200000,0.000000,0.0,0.00,0.2,0.1,0.00,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.1,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.000000,0.0,0.0


In [49]:
Scar_grouped.shape

(16, 56)

In [50]:
num_top_venues = 3

for hood in Scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scar_grouped[Scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0          Lounge  0.25
1  Breakfast Spot  0.25
2    Skating Rink  0.25


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2  American Restaurant   0.0


----Birch Cliff,Cliffside West----
                   venue  freq
0  General Entertainment  0.25
1           Skating Rink  0.25
2                   Café  0.25


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.14
1                Bakery  0.14
2                  Bank  0.14


----Clairlea,Golden Mile,Oakridge----
          venue  freq
0        Bakery   0.2
1      Bus Line   0.2
2  Intersection   0.1


----Clarks Corners,Sullivan,Tam O'Shanter----
                 venue  freq
0          Pizza Place   0.2
1  Fried Chicken Joint   0.1
2             Pharmacy   0.1


----Cliffcrest,Cliffside,Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Grouping Based on the Top ten venues

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scar_grouped['Neighborhood']

for ind in np.arange(Scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff,Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Electronics Store,Department Store
4,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Park,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Metro Station,Coffee Shop,Furniture / Home Store


Applying Clustering on the Dataset

In [53]:
# set number of clusters
kclusters = 5

Scar_grouped_clustering = Scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

In [54]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Scar_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scar_merged = Scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Scar_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3.0,History Museum,Bar,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Breakfast Spot,Intersection,Medical Center,Pizza Place,Rental Car Location,Mexican Restaurant,Electronics Store,Spa,Discount Store,Clothing Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Clothing Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Electronics Store,Department Store


In [55]:
mean=Scar_merged['Cluster Labels'].mean()
Scar_merged['Cluster Labels']=Scar_merged['Cluster Labels'].replace(np.nan,mean)

In [56]:
Scar_merged['Cluster Labels']  = Scar_merged['Cluster Labels'].astype('int64')

Map Obtained AferClustering

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scar_merged['Latitude'], Scar_merged['Longitude'], Scar_merged['Neighbourhood'], Scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Data Set Obtained After Clustering 

In [58]:
Scar_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3,History Museum,Bar,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1,Breakfast Spot,Intersection,Medical Center,Pizza Place,Rental Car Location,Mexican Restaurant,Electronics Store,Spa,Discount Store,Clothing Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Clothing Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Electronics Store,Department Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Playground,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,1,Discount Store,Department Store,Convenience Store,Coffee Shop,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,1,Bakery,Bus Line,Park,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Metro Station,Coffee Shop,Furniture / Home Store
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,4,American Restaurant,Motel,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


From Cluster 1,2 We can first common venues consists mostly of food related .So we can Conclude to have a meal there.

For Cluster 0 most common venue is Playground .So these places have good Playgrounds to play.

For Cluster 3 most common venue is Museum.So these places are famous for Museum.

For Cluster 4 most common venue is American Restuarant,Motel .so these places are famous to have breakfast in evenings.

These Analysis Obatained at Time of Around 3pm in India .So the results may change According to time